In [1]:
# STEP-1: LOAD LIBRARIES
# Standard library
import os
from pathlib import Path

# Third-party libraries
import duckdb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Custom osbad library for anomaly detection
import osbad.config as bconf
from osbad.database import BenchDB

In [2]:
# Path to database directory
DB_DIR = bconf.DB_DIR

db_filepath = DB_DIR.joinpath("train_dataset_severson.db")

In [3]:
# Create a DuckDB connection
con = duckdb.connect(
    db_filepath,
    read_only=True)

# Load all training dataset from duckdb
df_duckdb = con.execute(
    "SELECT * FROM df_train_dataset_sv").fetchdf()

# Get the cell index of training dataset
unique_cell_index_train = df_duckdb["cell_index"].unique()
print(f"Unique cell index: {unique_cell_index_train}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Unique cell index: ['2017-05-12_5_4C-50per_3C_CH13' '2017-05-12_5_4C-50per_3C_CH14'
 '2017-05-12_5_4C-60per_3C_CH15' '2017-05-12_5_4C-60per_3C_CH16'
 '2017-05-12_5_4C-70per_3C_CH17' '2017-05-12_5_4C-70per_3C_CH18'
 '2017-05-12_6C-40per_3C_CH25' '2017-05-12_6C-40per_3C_CH26'
 '2017-05-12_6C-50per_3C_CH27' '2017-05-12_6C-50per_3C_CH28'
 '2017-05-12_6C-60per_3C_CH29' '2017-05-12_6C-60per_3C_CH30'
 '2017-05-12_7C-40per_3C_CH37' '2017-05-12_7C-40per_3C_CH38'
 '2017-05-12_4C-80per_4C_CH5' '2017-05-12_4C-80per_4C_CH6'
 '2017-05-12_4_4C-80per_4_4C_CH7' '2017-05-12_5_4C-80per_5_4C_CH11'
 '2017-05-12_5_4C-80per_5_4C_CH12' '2017-05-12_3_6C-80per_3_6C_CH1'
 '2017-05-12_3_6C-80per_3_6C_CH2' '2017-05-12_3_6C-80per_3_6C_CH3'
 '2017-05-12_5_4C-40per_3_6C_CH19']


In [4]:
# Get the cell-ID from cell_inventory
selected_cell_label = "2017-05-12_5_4C-70per_3C_CH17"

# Create a subfolder to store fig output
# corresponding to each cell-index
selected_cell_artifacts_dir = bconf.artifacts_output_dir(
    selected_cell_label)

In [5]:
# -------------------------------------------------------------------------
# STEP-3: LOAD BENCHMARKING DATASET

# Import the BenchDB class
# Load only the dataset based on the selected cell
benchdb = BenchDB(
   db_filepath,
   selected_cell_label)

# load the benchmarking dataset
df_selected_cell = benchdb.load_benchmark_dataset(
   dataset_type="train")

if df_selected_cell is not None:

   filter_col = [
      "cell_index",
      "cycle_index",
      "discharge_capacity",
      "voltage"]

   # Drop true labels from the benchmarking dataset
   # and filter for selected columns only
   df_selected_cell_without_labels = benchdb.drop_labels(
      df_selected_cell,
      filter_col)

   # Extract true outliers cycle index from benchmarking dataset
   true_outlier_cycle_index = benchdb.get_true_outlier_cycle_index(
      df_selected_cell)
   print(f"True outlier cycle index:")
   print(true_outlier_cycle_index)

Database is found in the given filepath.
Loading benchmarking dataset now...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

****************************************************************************************************
True outlier cycle index:
[  0.  40. 147. 148.]


### Complete training data

In [6]:
df_duckdb

,test_time,cycle_index,cell_index,voltage,discharge_capacity,current,internal_resistance,temperature,outlier
0,1801.8327,0.0,2017-05-12_5_4C-50per_3C_CH13,3.291040,0.000016,-0.450999,0.021151,29.973852,1
1,1811.8329,0.0,2017-05-12_5_4C-50per_3C_CH13,3.281339,0.001570,-0.559847,0.021151,30.012213,1
2,1821.8363,0.0,2017-05-12_5_4C-50per_3C_CH13,3.277671,0.003125,-0.559859,0.021151,29.995052,1
3,1831.8373,0.0,2017-05-12_5_4C-50per_3C_CH13,3.275081,0.004680,-0.559828,0.021151,29.981087,1
4,1841.8468,0.0,2017-05-12_5_4C-50per_3C_CH13,3.273016,0.006237,-0.559821,0.021151,30.000351,1
...,...,...,...,...,...,...,...,...,...
12262709,456176.4267,879.0,2017-05-12_5_4C-40per_3_6C_CH19,2.000016,0.878140,-0.047946,0.017894,31.655296,1
12262710,456181.4292,879.0,2017-05-12_5_4C-40per_3_6C_CH19,1.999703,0.878207,-0.048657,0.017894,31.604851,1
12262711,456186.4310,879.0,2017-05-12_5_4C-40per_3_6C_CH19,2.000379,0.878273,-0.046559,0.017894,31.590973,1
12262712,456187.6726,879.0,2017-05-12_5_4C-40per_3_6C_CH19,1.999866,0.878289,-0.047244,0.017894,31.590973,1


### Describe the training dataset

In [7]:
df_duckdb.describe()

,test_time,cycle_index,voltage,discharge_capacity,current,internal_resistance,temperature,outlier
count,1.226271e+07,1.226271e+07,1.226271e+07,1.226271e+07,1.226271e+07,1.226271e+07,1.226271e+07,1.226271e+07
mean,1.620375e+05,3.350939e+02,2.806878e+00,6.429702e-01,-2.500158e+00,1.745635e-02,3.259392e+01,2.805245e-01
std,1.655052e+05,3.255462e+02,4.652349e-01,3.269458e-01,2.092206e+00,1.559968e-03,2.799349e+00,4.492555e-01
min,0.000000e+00,0.000000e+00,1.988390e+00,0.000000e+00,-5.178966e+00,0.000000e+00,2.390890e+01,0.000000e+00
25%,1.003909e+04,0.000000e+00,2.444114e+00,5.057269e-01,-4.400018e+00,1.658867e-02,3.009668e+01,0.000000e+00
50%,1.271887e+05,2.690000e+02,3.025914e+00,5.957790e-01,-4.399653e+00,1.698438e-02,3.209546e+01,0.000000e+00
75%,2.689684e+05,5.790000e+02,3.143797e+00,9.707202e-01,-1.100033e-01,1.751115e-02,3.460273e+01,1.000000e+00
max,9.103897e+05,1.226000e+03,4.623832e+00,2.884083e+00,-1.100785e-02,2.205313e-02,4.341938e+01,1.000000e+00


### Describe the data for selected cell

In [15]:
df_selected_cell.describe()

,test_time,cycle_index,voltage,discharge_capacity,current,internal_resistance,temperature,outlier
count,313326.000000,313326.000000,313326.000000,313326.000000,313326.000000,313326.000000,313326.000000,313326.000000
mean,179309.202977,331.490464,2.702667,0.697437,-3.262099,0.016591,33.592749,0.046613
std,106539.393974,206.385643,0.512590,0.436628,1.796975,0.001060,2.206655,0.210808
min,0.000000,0.000000,1.996414,0.000000,-4.408104,0.015552,25.144476,0.000000
25%,87235.563250,148.000000,2.028317,0.347776,-4.400076,0.015931,31.831541,0.000000
50%,175271.859550,328.000000,2.922445,0.818756,-4.399904,0.016098,33.575077,0.000000
75%,274878.008975,512.000000,3.117314,0.998362,-1.099984,0.017006,35.282305,0.000000
max,390923.695200,691.000000,4.623832,2.884083,-0.021661,0.021567,39.733044,1.000000


### Unique cells in training dataset

In [8]:
df_duckdb["cell_index"].unique()

array(['2017-05-12_5_4C-50per_3C_CH13', '2017-05-12_5_4C-50per_3C_CH14',
       '2017-05-12_5_4C-60per_3C_CH15', '2017-05-12_5_4C-60per_3C_CH16',
       '2017-05-12_5_4C-70per_3C_CH17', '2017-05-12_5_4C-70per_3C_CH18',
       '2017-05-12_6C-40per_3C_CH25', '2017-05-12_6C-40per_3C_CH26',
       '2017-05-12_6C-50per_3C_CH27', '2017-05-12_6C-50per_3C_CH28',
       '2017-05-12_6C-60per_3C_CH29', '2017-05-12_6C-60per_3C_CH30',
       '2017-05-12_7C-40per_3C_CH37', '2017-05-12_7C-40per_3C_CH38',
       '2017-05-12_4C-80per_4C_CH5', '2017-05-12_4C-80per_4C_CH6',
       '2017-05-12_4_4C-80per_4_4C_CH7',
       '2017-05-12_5_4C-80per_5_4C_CH11',
       '2017-05-12_5_4C-80per_5_4C_CH12',
       '2017-05-12_3_6C-80per_3_6C_CH1', '2017-05-12_3_6C-80per_3_6C_CH2',
       '2017-05-12_3_6C-80per_3_6C_CH3',
       '2017-05-12_5_4C-40per_3_6C_CH19'], dtype=object)

### Maximum number of cycles for each cell

In [9]:
df_duckdb.groupby('cell_index', observed=True)['cycle_index'].max()

cell_index
2017-05-12_3_6C-80per_3_6C_CH1     1189.0
2017-05-12_3_6C-80per_3_6C_CH2     1178.0
2017-05-12_3_6C-80per_3_6C_CH3     1176.0
2017-05-12_4C-80per_4C_CH5         1226.0
2017-05-12_4C-80per_4C_CH6         1226.0
2017-05-12_4_4C-80per_4_4C_CH7     1074.0
2017-05-12_5_4C-40per_3_6C_CH19     879.0
2017-05-12_5_4C-50per_3C_CH13       906.0
2017-05-12_5_4C-50per_3C_CH14       788.0
2017-05-12_5_4C-60per_3C_CH15       880.0
2017-05-12_5_4C-60per_3C_CH16       719.0
2017-05-12_5_4C-70per_3C_CH17       691.0
2017-05-12_5_4C-70per_3C_CH18       788.0
2017-05-12_5_4C-80per_5_4C_CH11     534.0
2017-05-12_5_4C-80per_5_4C_CH12     559.0
2017-05-12_6C-40per_3C_CH25        1017.0
2017-05-12_6C-40per_3C_CH26         854.0
2017-05-12_6C-50per_3C_CH27         860.0
2017-05-12_6C-50per_3C_CH28         917.0
2017-05-12_6C-60per_3C_CH29         731.0
2017-05-12_6C-60per_3C_CH30         757.0
2017-05-12_7C-40per_3C_CH37         704.0
2017-05-12_7C-40per_3C_CH38         648.0
Name: cycle_index, dtyp

### Test time for each training cell in hours

In [17]:
df_duckdb.groupby('cell_index', observed=True)['test_time'].max()/60/60

cell_index
2017-05-12_3_6C-80per_3_6C_CH1     252.885627
2017-05-12_3_6C-80per_3_6C_CH2     252.886021
2017-05-12_3_6C-80per_3_6C_CH3     252.414388
2017-05-12_4C-80per_4C_CH5         252.695361
2017-05-12_4C-80per_4C_CH6         252.622708
2017-05-12_4_4C-80per_4_4C_CH7     250.910616
2017-05-12_5_4C-40per_3_6C_CH19    126.718816
2017-05-12_5_4C-50per_3C_CH13      126.685187
2017-05-12_5_4C-50per_3C_CH14      123.048127
2017-05-12_5_4C-60per_3C_CH15      123.047467
2017-05-12_5_4C-60per_3C_CH16      109.085501
2017-05-12_5_4C-70per_3C_CH17      108.589915
2017-05-12_5_4C-70per_3C_CH18      123.033088
2017-05-12_5_4C-80per_5_4C_CH11    108.885955
2017-05-12_5_4C-80per_5_4C_CH12    108.965111
2017-05-12_6C-40per_3C_CH25        214.135535
2017-05-12_6C-40per_3C_CH26        122.865968
2017-05-12_6C-50per_3C_CH27        122.803122
2017-05-12_6C-50per_3C_CH28        122.917247
2017-05-12_6C-60per_3C_CH29        108.913145
2017-05-12_6C-60per_3C_CH30        122.859651
2017-05-12_7C-40per_3C_

### Calculate outlier cycles percentage for each cell

In [43]:
# 1) Reduce to one record per (cell_index, cycle_index),
#    where cycle_outlier = 1 if any row in that cycle has outlier==1
cycle_flags = (
    df_duckdb.groupby(['cell_index', 'cycle_index'], observed=True)['outlier']
      .max()                       # any 1 -> 1, else 0
      .reset_index(name='cycle_outlier')
)

print(cycle_flags.head())

per_cell_counts = (
    cycle_flags
    .groupby(['cell_index', 'cycle_outlier'], observed=True)
    .size()
    .unstack(fill_value=0)   # columns 0 and 1 with counts
    .rename(columns={0: 'n_cycles_non_outlier', 1: 'n_cycles_outlier'})
    .reset_index()
)

per_cell_counts["%_outlier"] = (
    (per_cell_counts["n_cycles_outlier"] * 100)/
    (per_cell_counts['n_cycles_non_outlier'] 
     + per_cell_counts['n_cycles_outlier'])
)

print(per_cell_counts)

print("Average % of outliers in each cell:", 
      per_cell_counts['%_outlier'].mean())

                       cell_index  cycle_index  cycle_outlier
0  2017-05-12_3_6C-80per_3_6C_CH1          0.0              1
1  2017-05-12_3_6C-80per_3_6C_CH1          1.0              0
2  2017-05-12_3_6C-80per_3_6C_CH1          2.0              0
3  2017-05-12_3_6C-80per_3_6C_CH1          3.0              0
4  2017-05-12_3_6C-80per_3_6C_CH1          4.0              0
cycle_outlier                       cell_index  n_cycles_non_outlier  \
0               2017-05-12_3_6C-80per_3_6C_CH1                  1185   
1               2017-05-12_3_6C-80per_3_6C_CH2                  1176   
2               2017-05-12_3_6C-80per_3_6C_CH3                  1174   
3                   2017-05-12_4C-80per_4C_CH5                  1222   
4                   2017-05-12_4C-80per_4C_CH6                  1221   
5               2017-05-12_4_4C-80per_4_4C_CH7                  1070   
6              2017-05-12_5_4C-40per_3_6C_CH19                   875   
7                2017-05-12_5_4C-50per_3C_CH13      

### completeness & missingness 

In [52]:
null_rate = df_duckdb.isna().mean().sort_values(ascending=False)
null_rate

test_time              0.0
cycle_index            0.0
cell_index             0.0
voltage                0.0
discharge_capacity     0.0
current                0.0
internal_resistance    0.0
temperature            0.0
outlier                0.0
dtype: float64

In [32]:
issues = {}

# a) Duplicate rows (exact duplicates)
issues['duplicate_rows'] = int(df_duckdb.duplicated().sum())

# b) Duplicate keys within a cycle (same timestamp twice)
issues['dup_time_within_cycle'] = int(
    df_duckdb.duplicated(subset=['test_time','cycle_index','cell_index']).sum()
)

# c) Invalid outlier flag (should be 0/1/NaN)
bad_outlier = ~df_duckdb['outlier'].isin([0,1,np.nan])
issues['bad_outlier_values'] = int(bad_outlier.sum())

# d) Negative or zero internal resistance
issues['negative_or_zero_R_rows'] = int(
    (df_duckdb['internal_resistance']<=0).sum()
    )

#e) Count rows where voltage is outside [0.1, 4.5]
issues['voltage outside chem limits'] = int(
    ((df_duckdb['voltage']<0.1) | (df_duckdb['voltage']>4.5)).sum()
)

# f) Temperature outside safe range (-40°C to 90°C).
issues['temp outside safe limits'] = int(
    ((df_duckdb['temperature']<-40) | (df_duckdb['temperature']>90)).sum()
)

# g) Capacity negative
issues['negative_capacity'] = int(
    (df_duckdb['discharge_capacity']<0).sum()
    )

# h) Non-finite values
issues['non_finite_any'] = int(
    ~np.isfinite(df_duckdb.select_dtypes(include=[float,int]).to_numpy()).all()
    )

issues

{'duplicate_rows': 0,
 'dup_time_within_cycle': 3,
 'bad_outlier_values': 0,
 'negative_or_zero_R_rows': 40,
 'voltage outside chem limits': 103,
 'temp outside safe limits': 0,
 'negative_capacity': 0,
 'non_finite_any': 0}

### size of the training dataset 

In [ ]:
# Assuming you have a DataFrame called df
size_bytes = df_duckdb.memory_usage(deep=True).sum()

# Convert to human-readable format
def human_readable_size(size, decimal_places=2):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.{decimal_places}f} {unit}"
        size /= 1024

print(f"Size of DataFrame: {human_readable_size(size_bytes)}")

Size of DataFrame: 44.82 MB
